<a href="https://colab.research.google.com/github/Daniel8B/arch-style/blob/main/ArchStyleApp_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [ ]:
path = Path()
learn_inf = load_learner(path/'export.pkl', cpu=True)

This has returned three things: 
* the predicted category in the same format you originally provided (in this case that's a string)
* the index of the predicted category
* the probabilities of each category. 

The last two are based on the order of categories in the `vocab` of the DataLoaders; that is, the stored list of all possible categories.

In [ ]:
learn_inf.dls.vocab # index 1 which referts to 'modernist'

To create a working web app using only Jupyter notebooks we only need:
* IPython widgets (ipywidgets) - IPython widgets are GUI components that bring together JavaScript and Python functionality in a web browser, and can be created and used within a Jupyter notebook. For instance, the image cleaner that we saw earlier in this chapter is entirely written with IPython widgets. **However, we don't want to require users of our application to run Jupyter themselves.**

* Voilà - That is why Voilà exists. It is a system for making applications consisting of IPython widgets available to end users, without them having to use Jupyter at all. Voilà is taking advantage of the fact that a notebook already is a kind of web application, just a rather complex one that depends on another web application: Jupyter itself. Essentially, it helps us automatically convert the complex web application we've already implicitly made (the notebook) into a simpler, easier-to-deploy web application, which functions like a normal web application rather than like a notebook.

But we still have the advantage of developing in a notebook, so with ipywidgets, we can build up our GUI step by step. We will use this approach to create a simple image classifier. First, we need a file upload widget:

In [ ]:
# File uploader widget
btn_upload = widgets.FileUpload()
btn_upload # the location of the uploaded file/s in memory

In [ ]:
# Convert the uploaded image from bytes to PILImage
img = PILImage.create(btn_upload.data[-1])

In [ ]:
# Display the image with Output widget
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
"""Since I couldn't use the `predict` function below with the PILImage, the workaround
I found was by saving the image from memory into Colab"""

from PIL import Image
    
for name, file_info in btn_upload.value.items():
    img = Image.open(io.BytesIO(file_info['content']))# open the image from bytes
        
img.save(name) # save the image as "jpeg" on Colab

In [ ]:
# Make prediction, save index, save probability of prediction
pred,pred_idx,probs = learn_inf.predict(f"/content/{name}")

In [ ]:
#use `Label` to display the prediction and its probability
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
# Button to initialize the classification after uploading an image
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
"""We'll also need a click event handler; that is, a function that will be called when it's pressed."""

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(f"/content/{name}")
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [ ]:
VBox([widgets.Label('Select your style!'), 
      btn_upload, btn_run, out_pl, lbl_pred])